# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686585


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:05<01:24,  2.91s/it]

Rendering models:  10%|▉         | 3/31 [00:07<01:10,  2.53s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<00:49,  1.83s/it]

Rendering models:  19%|█▉        | 6/31 [00:07<00:32,  1.31s/it]

...

Rendering models:  65%|██████▍   | 20/31 [00:09<00:01,  6.83it/s]

Rendering models:  74%|███████▍  | 23/31 [00:09<00:00,  8.49it/s]

Rendering models:  81%|████████  | 25/31 [00:09<00:00,  8.48it/s]

Rendering models:  87%|████████▋ | 27/31 [00:10<00:00,  8.23it/s]

Meckert7597                           0.001090
not-logged-in-0ed3d164fd32cee0b7f7    0.000910
CThomas                               0.000971
Camelron                              0.000749
SavinSpaceEngineer                    0.000587
not-logged-in-2159616075e1803d6325    0.004637
djswanso                              0.001249
austingasbarra                        0.002360
klmasters                             0.000441
kjkavene                              0.000612
not-logged-in-7aef656f86917a55b5e6    0.030092
not-logged-in-5f30b2bb9b7f9d49da41    0.017134
relseidy                              0.002906
crush202020                           0.025192
Planetme                              0.001262
Tacocat1127                           0.002642
pangeli5                              0.001351
acapirala                             0.001299
chavez01                              0.001597
Lavadude                              0.002275
clars915                              0.001201
entzo        

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())